In [75]:
import pandas as pd

clientes = pd.read_excel('base.xlsx', header=1, sheet_name='CLIENTES')
clientes.columns = clientes.columns.str.lower()
faturamento = pd.read_excel('base.xlsx', header=1, sheet_name='FATURAMENTO ')
# faturamento = faturamento.rename(columns={'Cliente_id' : 'cliente_id')}
churn = pd.read_excel('base.xlsx', header=1, sheet_name='CHURN').rename(columns={'mês_ano':'mes_ano'})

clientes.columns = clientes.columns.str.lower()
faturamento.columns = faturamento.columns.str.lower()
churn.columns = churn.columns.str.lower()


In [ ]:
df_completo = pd.merge(faturamento, clientes, on='cliente_id', how='left')

df_completo.columns = df_completo.columns.str.lower()
df_completo = df_completo.drop(columns='unnamed: 0_x')
df_completo

## Criar uma Tabela Dinâmica que mostre o faturamento mensal por segmento e por estado.

In [ ]:
tabela_dinamica = pd.pivot_table(
     df_completo,
     values='r$ receita',
     index=['segmento', 'estado'],
     columns=['mes_ano'],
     aggfunc='sum',
     fill_value=0,
     margins=True,
     margins_name='TOTAL'
)
tabela_dinamica

##  Liste os 10 clientes que tiveram maior faturamento no último mês

In [ ]:
ultimo_mes = faturamento['mes_ano'].max()

faturamento_ultimo_mes = faturamento[faturamento['mes_ano'] == ultimo_mes].copy()

df_ultimo_mes = pd.merge(
    faturamento_ultimo_mes,
    clientes,
    on='cliente_id',
    how='left'
)
df_ultimo_mes.columns = df_ultimo_mes.columns.str.lower()

top10 = df_ultimo_mes.sort_values(by='r$ receita', ascending=False).head(10)

resultado = top10[['mes_ano', 'estado', 'r$ receita', 'cliente_id', 'segmento']]

resultado

## Calcule a margem bruta por cliente no último mês (margem_bruta = (receita - custo) / receita).

In [ ]:
df_margem = pd.merge(
    faturamento,
    clientes,
    on='cliente_id',
    how='left'
)
df_margem.columns = df_margem.columns.str.lower()

df_margem['margem'] = (df_margem['r$ receita'] - df_margem['r$ custo']).round(2)
df_margem['margem_%'] = (df_margem['margem'] / df_margem['r$ receita'] * 100).round(2)
df_margem


## Identifique a taxa de churn por segmento de empresa no último mês.


In [ ]:
ultimo = churn['mes_ano'].max()
meses = sorted(churn['mes_ano'].unique())
anterior = meses[meses.index(ultimo) - 1] if meses.index(ultimo) > 0 else None

if anterior:
    ativos = (churn[(churn['mes_ano'] == anterior) & (churn['status'] == 'Ativo')]
    .merge(clientes, on='cliente_id')
    .groupby('segmento').size())
else:
    ativos = clientes.groupby('segmente').size()
    